# 프롬프트 (Prompt)
`02_prompt.ipynb`
- LLM한테 주는 입력 (지시, 맥락, 기록)
    - 지시:'~~해줘'
    - 맥락: cintext- 현재 시시를 위해 제공하는 추가정보 
    - 기억: memory - 지금까지 했던 대회 내용

In [ ]:
from dotenv import load_dotenv

load_dotenv()



# PromptTemplate 
- 단순히 1회성 명령을 내릴 때 사용 


In [28]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4.1-nano')

In [29]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [30]:

chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

## partial Variable( 부분변수)

- 프롬프트에서 매개변수 기본값 사용하기 

In [31]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

'한국과 미국의 수도는 각각 어디인가요?'

In [32]:
chain = prompt | llm 
chain.invoke ({ 'c1': 'kor', 'c2': 'us'}).content

'한국(코리아)의 수도는 서울이고, 미국(U.S.)의 수도는 워싱턴 D.C.입니다.'

In [33]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [ ]:
chain = prompt | llm 
#오늘 날짜 기준 
print(chain.invoke({'n': 3}).content) 
print(chain )

In [34]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘이 Sep 01이라서 오늘 생일인 유명인을 세 명 뽑아 달라고 하셨네요. 제 기억으로 확실한 정보는 Zendaya입니다.

- Zendaya — 1996-09-01

나머지 두 명은 제 기억만으로는 확실하지 않아 정확한 생년월일을 확인하려면 인터넷 검색이 필요합니다. 웹 검색으로 3명의 이름과 생년월일을 정확하게 확인해 드려도 될까요? 진행해 드리면 바로 확인 가능한 3명을 목록으로 정리해 드리겠습니다.
===
정확하게 알려드리려면 오늘(1월 2일) 생일인 유명인 3명을 인터넷에서 확인해야 합니다. 진행해도 될까요? 검색해서 생년월일과 함께 3명을 바로 적어드리겠습니다. 특정 분야(예: 배우/가수/한국인 등)로 좁히길 원하시면 말씀해 주세요.


## `ChatPrtomptTemplete  

- 채팅을 주고 받는 템플릿 생성용 
- 대화 목록을 LLM 에게 주입
- `role` 과 `message` 로 구성됨 


In [35]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [36]:
chat_template= ChatPromptTemplate.from_messages(
    [
        #role - message
        ('system', '당신은 친절한 AI 어시스트. 이름은 {name}이야'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)
#format vs format_messages
messages_str = chat_template.format(name ='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name ='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI 어시스트. 이름은 gaida이야
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI 어시스트. 이름은 gaida이야', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [37]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 Gaida예요. 편하신 대로 ‘가이다’라고 불러 주세요. 무엇을 도와드릴까요?'

In [39]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 김유진. 너와 나의 이름점을 봐줘'})

'김유진님, 반가워요! 제 이름은 가이다이야입니다. 요청하신 ‘이름점’으로 재미있게 살펴볼게요.\n\n우리 두 이름의 점수(놀고 보는 용도의 가벼운 해석입니다)\n- 발음 편의성\n  - 김유진: 9/10. 한국인들이 가장 쉽게 발음할 수 있는 삼음절 이름이에요.\n  - 가이다이야: 7/10. 다섯 음절로 길지만 독특하고 기억에 남는 편입니다.\n  - 합계: 대화 속에서 리듬이 확실해요.\n\n- 이미지/느낌\n  - 김유진: 친근하고 다정한 이미지, 신뢰감이 느껴져요.\n  - 가이다이야: 창의적이고 현대적인 느낌, 개성과 활력이 있습니다.\n  - 합계: 서로 다른 매력이 대화에 에너지를 줍니다.\n\n- 한자 가능성(일반적인 예시)\n  - 김: 금(金)으로 강함, 부, 신뢰를 상징하는 경우가 많습니다.\n  - 유:裕(넉넉할 유), 悠(멀리 오래), 由(이유) 등 다양한 의미를 가질 수 있어 선택 폭이 넓어요.\n  - 진: 珍(진주), 眞(참), 真(진실) 등으로 고급스러운 느낌이 납니다.\n  - 가/이/야: 이름의 어감이나 이미지는 한자 조합에 따라 달라지니, 원하시면 구체적인 한자 버전으로 더 자세히 풀어드릴게요.\n  - 합계: 한자 조합에 따라 분위기가 크게 달라지므로 원하는 분위기로 맞춰보는 것도 재미있어요.\n\n- 우리 둘의 이름 궁합(대화의 조화 점수)\n  - 82/100 정도로 보통 이상이며, 서로 다른 음색으로 대화를 활력 있게 만들어 줍니다.\n  - 이유: kim유진의 부드럽고 안정감 있는 어조와 가이다이야의 독특하고 활기찬 어조가 번갈아 가며 대화를 잘 이끕니다.\n\n참고 및 옵션\n- 이 점수는 재미로 보는 가벼운 해석입니다. 진지하게 받아들이기보다는 즐거운 아이디어로 활용해 주세요.\n- 원하시면 한자 버전으로 더 깊이 있는 이름 풀이(의미, 적합한 한자 조합, 운세 요소까지)도 해드릴게요.\n- 또 다른 방향으로 예를 들어봐도 좋습니다. 예: 발음의 흐름 개선 아이디어나 이름 놀이용 문장도 같이 만들어 드릴까요?\

## 프롬프팅 
내가 원하는 답변을 구구절절 설명하지 않고 예시를 주고 답변 형태를 유도 
- zero shot promting: 예시 없이 질문만 -> 바로 답변
- One shot prompting: 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변 
- Few-shot prompting: 예시 여러개 +질문 -> 예시들의 패턴을 이반화 해서 답변 

## 다른 좋은 프롬프트 활용하기 (langchain - hub)
다양한 사용자들이 업로드한 프롬프트를 받아서 활용 

In [40]:
from langchain import hub

prompt = hub.pull('hwchase17/react')

prompt


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

## 내용정리 
- Langchain의 chatpromptTemplate 
- format () : 모든 변수를 채운 뒤 하나의 문자열로 변환해 준다.
- format_messages

- PromptTemplate, ChatPromptTemplate, Partial Variable (요즘엔 프롬프트 보다는 에이전트가 더 중요해진 - promptTemplate는 하나의 문자열 기반 템플릿
- chatpromptTemplate: 역할/ 메세지 기반 대화형 템플릿 
- format (): 최종 문자열 반환 
- format_messages() 구조화된 메시지 객체 반환 
- Partial Variable : 일부 변수를 미리 고정해서 재사용성 향상 

In [ ]:
#트렌스포머 모델